In [ ]:
%%{init: {'theme': 'forest'}}%%
graph TD
    A[Initial Query] --> B{Graph DB Router}
    B -->|Use Graph DB| C[Query Graph DB]
    C --> D[Expand Queries]
    D --> E[Subgraph Router]
    B -->|Skip Graph DB| F[Refine Query]
    F --> E
    
    E --> G[ECT Subgraph]
    E --> H[AC Subgraph]
    E --> I[... More Subgraphs]
    
    G & H & I --> J[Result Aggregation]
    J --> K[Final Response Generation]
    
    style A fill:#f9f,stroke:#333,stroke-width:2px
    style B fill:#bbf,stroke:#f66,stroke-width:2px,color:#fff
    style C fill:#bfe,stroke:#333,stroke-width:2px
    style D fill:#bfe,stroke:#333,stroke-width:2px
    style E fill:#fbb,stroke:#666,stroke-width:2px
    style F fill:#feb,stroke:#333,stroke-width:2px
    style G fill:#e0e0e0,stroke:#999,stroke-width:2px
    style H fill:#e0e0e0,stroke:#999,stroke-width:2px
    style I fill:#e0e0e0,stroke:#999,stroke-width:2px
    style J fill:#bfb,stroke:#333,stroke-width:2px
    style K fill:#f9f,stroke:#333,stroke-width:2px
    
    %% Edge styles
    linkStyle 1 stroke:#0f0,stroke-width:2px;
    linkStyle 2 stroke:#0f0,stroke-width:2px;
    linkStyle 3 stroke:#0f0,stroke-width:2px;
    linkStyle 4 stroke:#f90,stroke-width:2px;
    linkStyle 5 stroke:#f90,stroke-width:2px;